<a href="https://colab.research.google.com/github/abay-qkt/gps-to-photos-from-google-timeline/blob/main/set_gps_to_photos_from_google_timelineipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
path = "location-history.json"

# 準備

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

## タイムラインデータのロード

In [ ]:
df = pd.read_json(path)
df["tz_info"]=df["startTime"].str[23:]
df["endTime"] = pd.to_datetime(df["endTime"].str[:23])
df["startTime"] = pd.to_datetime(df["startTime"].str[:23])
df.set_index(["endTime","startTime","tz_info"],inplace=True)

# timelinePath
tp = df["timelinePath"].dropna()
tp = pd.json_normalize(tp.explode()).set_index(tp.explode().index)
tp[["path_lat","path_lon"]] = tp["point"].str[4:].str.split(",",expand=True).astype(float)
tp = tp.drop(["point"],axis=1).reset_index()
tp["durationMinutesOffsetFromStartTime"]=tp["durationMinutesOffsetFromStartTime"].astype(int)
tp["pointTime"]=(
    tp["startTime"]  #  startTime+durationMinutesOffsetFromStartTime時点でその緯度経度に居たと解釈する
    +pd.to_timedelta(tp["durationMinutesOffsetFromStartTime"],unit='minute')
    +pd.to_timedelta(9,unit='hour')  # 日本時間に変換する
)
tp["prevTime"]=tp["pointTime"].shift()  # 直前の時刻を示す列も用意しておく

# timelineMemory
tm = df["timelineMemory"].dropna()
tm = pd.json_normalize(tm).set_index(tm.index).explode("destinations")
tm["destinations"]=tm["destinations"].map(lambda x: x["identifier"])
tm["distanceFromOriginKms"]=tm["distanceFromOriginKms"].astype(int)
tm.reset_index(inplace=True)

# activity
ac = df["activity"].dropna()
ac = pd.json_normalize(ac).set_index(ac.index)
ac[["start_lat","start_lon"]]=ac["start"].str[4:].str.split(",",expand=True).astype(float)
ac[["end_lat","end_lon"]]=ac["end"].str[4:].str.split(",",expand=True).astype(float)
ac.drop(["start","end"],axis=1,inplace=True)
ac["distanceMeters"]=ac["distanceMeters"].astype(float)
ac.reset_index(inplace=True)

# visit
vt = df["visit"].dropna()
vt = pd.json_normalize(vt).set_index(vt.index)
vt[["place_lat","place_lon"]]=vt["topCandidate.placeLocation"].str[4:].str.split(",",expand=True).astype(float)
vt.drop(["topCandidate.placeLocation"],axis=1,inplace=True)
vt["hierarchyLevel"]=vt["hierarchyLevel"].astype(int)
vt["topCandidate.probability"]=vt["topCandidate.probability"].astype(float)
vt.reset_index(inplace=True)

In [ ]:
# 訪問地ごとに集計
vt_info = vt.copy()
vt_info["startDate"]=vt_info["startTime"].dt.date.astype(str)
vt_info = (
    vt_info
    .groupby(["topCandidate.placeID","place_lat","place_lon"])["startDate"]
    .agg(["nunique","max"])
    .sort_values("max")
    .rename(columns={"nunique":"visit_count","max":"last_visit"})
    .reset_index()
)

# exif

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from PIL.ExifTags import Base,GPS,IFD,TAGS,GPSTAGS
from tqdm import tqdm

use_tag_ids = {
    "0th IFD":[
        Base.DateTime,
        Base.Model,
        Base.Software,
        Base.Orientation
    ],
    "Exif IFD":[
        Base.DateTimeOriginal,
        Base.DateTimeDigitized,
        Base.SubsecTime,
        Base.SubsecTimeOriginal,
        Base.SubsecTimeDigitized,
        Base.FNumber,
        Base.ExposureTime,
        Base.ISOSpeedRatings,
        Base.FocalLength,
        Base.FocalLengthIn35mmFilm,
        Base.ExposureProgram,
        Base.SceneCaptureType,
        Base.LensModel
    ],
    "GPS Info IFD":[
        GPS.GPSLatitude,
        GPS.GPSLatitudeRef,
        GPS.GPSLongitude,
        GPS.GPSLongitudeRef
    ]
}

MODE_DICT = {
    "ExposureProgram":{
        0:"未定義",
        1:"マニュアル",
        2:"ノーマルプログラム",
        3:"絞り優先",
        4:"シャッター優先",
        5:"creativeプログラム",  # 被写界深度方向にバイアス
        6:"actionプログラム",  # シャッタースピード高速側にバイアス
        7:"ポートレイトモード",  # クローズアップ撮影、背景フォーカス外す
        8:"ランドスケープモード",  # landscape撮影、背景はフォーカス合う
    },
    "SceneCaptureType":{
        0:"標準",
        1:"風景",
        2:"人物",
        3:"夜景"
    }
}

def load_exif(path):
    with Image.open(path) as im:
        exif = im.getexif()
    # 各IFDの情報を必要なタグだけ取得
    zeroth_ifd = {TAGS[tag_id]: value for tag_id, value in exif.items()
                        if tag_id in use_tag_ids["0th IFD"]}
    exif_ifd = {TAGS[tag_id]: value for tag_id, value in exif.get_ifd(IFD.Exif).items()
                        if tag_id in use_tag_ids["Exif IFD"]}
    gps_ifd = {GPSTAGS[tag_id]: value for tag_id, value in exif.get_ifd(IFD.GPSInfo).items()
                        if tag_id in use_tag_ids["GPS Info IFD"]}
    exif_dict = dict(**zeroth_ifd,**exif_ifd,**gps_ifd) # 辞書の連結
    exif_dict["path"] = path

    return exif_dict

def convert_exif_cols(exif_df):
    # datetime型への変換
    time_subsec = [("DateTime",         "SubsecTime"),  # 日付系のカラム名と対応するSubsec(ミリ秒情報)のカラム名
                    ("DateTimeOriginal", "SubsecTimeOriginal"),
                    ("DateTimeDigitized","SubsecTimeDigitized")]
    for time,subsec in time_subsec:
        if subsec in exif_df.columns: # ミリ秒情報があれば日付情報にマージしdatetime化
            exif_df[time] = exif_df[time].astype(str).replace("nan","")+"."\
                            +exif_df[subsec].astype(str).replace("nan","0")# 日付と小数点以下を"."で連結
            exif_df[time] = exif_df[time].replace(".0",np.nan) # 日付自体が欠損の場合↑の処理によって".0"だけになるので欠損にする
            exif_df[time] = pd.to_datetime(exif_df[time],format='%Y:%m:%d %H:%M:%S.%f',errors='coerce')
        elif time in exif_df.columns: # なければそのままdatetime化
            exif_df[time] = pd.to_datetime(exif_df[time],format='%Y:%m:%d %H:%M:%S')

    exif_df["FocalLength"] = exif_df["FocalLength"].astype(float)
    exif_df["FNumber"] = exif_df["FNumber"].astype(float)
    exif_df["ShutterSpeed"] = exif_df["ExposureTime"].map(lambda x:str(x.real)) # 分数表記
    exif_df["ExposureTime"] = exif_df["ExposureTime"].astype(float) # 数値

    # カテゴリ情報をカラムに関して、番号をカテゴリ名に変換
    for key in MODE_DICT.keys():
        if(key in exif_df.columns):
            exif_df[key] = exif_df[key].map(MODE_DICT[key])

    # GPS情報の変換（度分秒のタプル→度）
    if "GPSLatitude" in exif_df.columns:
        exif_df["GPSLatitude"]  = exif_df["GPSLatitude"].map(dms2deg).astype(float)
        exif_df["GPSLongitude"] = exif_df["GPSLongitude"].map(dms2deg).astype(float)
        exif_df["GPSLatitude"]  =  exif_df["GPSLatitude"]*exif_df["GPSLatitudeRef"].replace("N",1).replace("S",-1).replace(0,np.nan)
        exif_df["GPSLongitude"] =  exif_df["GPSLongitude"]*exif_df["GPSLongitudeRef"].replace("E",1).replace("W",-1).replace(0,np.nan)

    # 欠損の場合0が入るみたいなので改めて欠損に置換
    int_cols = ["FocalLength","FocalLengthIn35mmFilm"]
    exif_df[int_cols] = exif_df[int_cols].replace(0,pd.NA)
    float_cols = ["FNumber","ExposureTime"]
    exif_df[float_cols] = exif_df[float_cols].replace(0,np.nan)

    return exif_df

# GPSデータの処理に使用
def dms2deg(x):
    # 緯度経度の度分秒フォーマットを度に変換
    return x[0]+x[1]/60+x[2]/3600 if type(x)==tuple else np.nan

def categorize_focal_length(x):
    # 参考
    # https://ptl.imagegateway.net/contents/original/glossary/標準レンズ、広角レンズ、望遠レンズ.html
    # https://av.jpn.support.panasonic.com/support/dsc/knowhow/knowhow21.html
    # https://goopass.jp/magazine/300mmsupertelephotoens10/
    if(pd.isna(x)):
        return np.nan
    elif(x<24):
        return "超広角(～23mm)"
    elif(x<35):
        return "広角(24～34mm)"
    elif(x<100):
        return "標準(35～99mm)"
    elif(x<300):
        return "望遠(100～299mm)"
    elif(x>=300):
        return "超望遠(300～mm)"

def categorize_exposure_time(x):
    if(pd.isna(x)):
        return np.nan
    elif(x<=1/1000):
        return "～1/1000sec"
    elif(x<1):
        return "1/800～1/10sec"
    elif(x>=1):
        return "1/8～sec"

def categorize_f_number(x):
    # 参考
    # https://photobook.ikuji-park.com/f-number.html
    if(pd.isna(x)):
        return np.nan
    elif(x<4):
        return "～F3.5"
    elif(x<8):
        return "F4～F7.1"
    elif(x<13):
        return "F8～F11"
    elif(x>=13):
        return "F13～"

def add_extra_cols(exif_df):
    exif_df["FocalLengthCategory"] = exif_df["FocalLengthIn35mmFilm"].map(categorize_focal_length)
    exif_df["ExposureTimeCategory"] = exif_df["ExposureTime"].map(categorize_exposure_time)
    exif_df["FNumberCategory"] = exif_df["FNumber"].map(categorize_f_number)
    return exif_df

def get_exif_df(path_list):
    # exif_dict_list = [load_exif(path) for path in tqdm(path_list)]
    exif_dict_list = [load_exif(path) for path in path_list]
    exif_df = pd.DataFrame(exif_dict_list)
    exif_df = convert_exif_cols(exif_df) # 型変換
    exif_df = add_extra_cols(exif_df) # カテゴリカラム追加
    return exif_df

# 既存のexif_dfがあれば、path_listからはまだ存在しないpathだけ読み込んで追加する
def get_exif_df_add(path_list,existing_exif_df=None):
    additional_path_list = sorted(set(path_list)-set(existing_exif_df["path"]))
    additional_exif_df = get_exif_df(additional_path_list)
    exif_df = pd.concat([existing_exif_df,additional_exif_df],ignore_index=True)
    return exif_df

# 写真の緯度軽度を計算

In [ ]:
# 写真を保存しているフォルダ
photo_dir_path = "2025-08-26"

In [ ]:
path_list = list(Path(photo_dir_path).glob("**/*.JPG")) # jpegファイル一覧を取得

In [ ]:
exif_df = get_exif_df(path_list) # Exifのデータフレームを取得

In [ ]:
# Googleタイムラインの時刻
timeline_time_list = tp["pointTime"].drop_duplicates().sort_values().to_list()
# exifの時刻
exif_time_list = exif_df["DateTimeOriginal"].drop_duplicates().sort_values().to_list()
# 両方の時刻
both_time_list = pd.Series(timeline_time_list+exif_time_list).drop_duplicates().sort_values().to_list()

In [ ]:
interpolated_latlon = (
    tp[["pointTime","path_lat","path_lon"]]
    .drop_duplicates(subset=["pointTime"],keep='last') # 重複する時刻を削除
    .set_index("pointTime") # Googleタイムラインの時刻をindexにする
    .reindex(both_time_list) # exifの時刻もindexに追加。緯度軽度は欠損で入る
    .interpolate(method="time") # 緯度軽度の欠損を時間で線形補間
    .loc[exif_time_list,:] # exifの時刻だけ残す
)

In [ ]:
# 時刻をキーに緯度経度を付与
exif_gps_df = pd.merge(
    exif_df[["path","DateTimeOriginal"]],
    interpolated_latlon.reset_index().rename(columns={"pointTime":"DateTimeOriginal"}),
    on='DateTimeOriginal',
    how='left'
)

# 写真に緯度軽度を書き込む

In [ ]:
# exiftoolを使って、緯度軽度情報を写真に書き込む
import subprocess
import glob
import os
from typing import Optional

def write_gps_exif_exiftool(
    file_path: str, lat: float, lon: float, alt: Optional[float] = None
):
    """
    exiftool で JPEG/RAW/HEIC 等にGPSを書き込む
    """
    cmd = [
        "exiftool",
        f"-GPSLatitude={lat}",
        f"-GPSLatitudeRef={'N' if lat >= 0 else 'S'}",
        f"-GPSLongitude={lon}",
        f"-GPSLongitudeRef={'E' if lon >= 0 else 'W'}",
    ]

    if alt is not None:
        cmd.append(f"-GPSAltitude={alt}")

    # 上書き
    cmd.append("-overwrite_original")

    # 書き込み対象
    cmd.append(file_path)

    result = subprocess.run(
                cmd,
                check=True,
                capture_output=True,
                text=True
    )
    return result


In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
result = exif_gps_df.progress_apply(lambda x:write_gps_exif_exiftool(x["path"],x["path_lat"],x["path_lon"]),axis=1)

100%|█████████████████████████████████████████████████████████████████████████| 410/410 [00:50<00:00,  8.12it/s]
